# ph recognition based on the image color
 ***
 
 ### I downloaded ph-scale images from [link](https://www.shutterstock.com/search/ph+scale). Then I split every image to 15 small ones and I labeled it based on the ph

In [19]:
import cv2 # for images
import matplotlib.pyplot as plt
import pickle
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn import svm
from sklearn.externals import joblib

pd.set_option('display.max_columns', None)

data_dir = 'C:\\Users\\asus\\Desktop\\data_science_python\\ph_project\\ph_datasets\\'

In [108]:
PH_COUNT = 15
images_list = []
PH_DATASETS = 7
# add image to list in format [blue, green, red, label=ph]
for j in range(PH_DATASETS):
    for i in range(PH_COUNT):
        image = cv2.imread(data_dir + 'ph{}\\{}.jpg'.format(j+1, i))
        images_list.append([image[3][3][0],image[3][3][1],image[3][3][2],i])

# transform data to pandas DataFrame
df = pd.DataFrame(np.asarray(images_list),columns=['blue','green','red','label'])
df = shuffle(df)
# df.head(-1)

In [107]:
y = np.array(df['label']) 
X = np.array(df.drop(['label'], axis=1))
print(y)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14  0  1  2  3  4  5  6  7  8
  9 10 11 12 13 14  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14  0  1  2
  3  4  5  6  7  8  9 10 11 12 13 14  0  1  2  3  4  5  6  7  8  9 10 11
 12 13 14  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14  0  1  2  3  4  5
  6  7  8  9 10 11 12 13 14]


# test and train data 

In [84]:
SIZE = X.shape[0]
TRAIN_SIZE = int(SIZE*0.9)
train_x = X[:TRAIN_SIZE]
train_y = y[:TRAIN_SIZE]
test_x = X[TRAIN_SIZE:]
test_y = y[TRAIN_SIZE:]
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((94, 3), (11, 3), (94,), (11,))

# Testing machine learning algorithms

***


#### GradientBoostingRegressor

In [92]:
from sklearn import tree
from sklearn.ensemble import GradientBoostingRegressor
# clf = GradientBoostingRegressor()
# clf.fit(train_x, train_y)

#### RandomForestRegressor

In [109]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
clf = RandomForestRegressor()
clf.fit(train_x, train_y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

#### GradientBoostingRegressor with tuning

In [156]:
clf = GradientBoostingRegressor(loss='lad', learning_rate=0.2,
                               max_depth=4,)
clf.fit(train_x, train_y)

ValueError: max_features must be in (0, n_features]

In [154]:
from sklearn.metrics import r2_score
# testing 
accuracy = 0
for i in range(len(test_x)):
    y_predict = clf.predict([test_x[i]])
    print('Predict {:.2f}, labels {}'.format(y_predict[0],test_y[i]))
    # add some margin
    if(abs(y_predict[0] - test_y[i]) <= 0.3):
        accuracy += 1
accuracy = float(accuracy/len(test_x))
print(accuracy)
# return the mean accuracy on the given test data and labels
r2 = r2_score(test_y, clf.predict(test_x))
print(r2)
# overfitting !

Predict 12.84, labels 14
Predict 1.00, labels 1
Predict 1.00, labels 1
Predict 13.15, labels 14
Predict 1.69, labels 2
Predict 6.90, labels 7
Predict 1.97, labels 3
Predict 12.70, labels 12
Predict 8.41, labels 9
Predict 11.68, labels 12
Predict 12.70, labels 12
0.2727272727272727
0.9834026776421096


In [29]:
# save model 
joblib.dump(clf,'saved_models//gradient_regression_85.pickle')

['gradient_regression_85.pickle']

In [155]:
# pickle save model
number = 2
model_pickle = open('saved_models//model-{:.2f}-v{}.pickle'.format(r2, number),'wb')
pickle.dump(clf, model_pickle)
model_pickle.close()